Tape measure
aruco tags
tripod
checkboard

Steps: 
Get a video - Together
Calibration - Satchel
Create a mask for finding ball - Zaraius
Create a yolo model for finding ball -  Satchel
Use mask for find position - Zaraius
Use bounding box for finding position - Zaraius
Connect frame by frame position to get velocity (SORT) - Satchel

Find size to depth research
hough transform
Once you've detected center of the ball, edge segmentation or line detector

In [ ]:
# Calibration
distance = []
ball_diam = []

In [ ]:
import cv2
import os
import numpy as np

class BallMeasurement:
    def __init__(self):
        self.points = []
        self.current_image = None
        self.window_name = "Measure Ball"
        self.diameter = None
        
    def mouse_callback(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            # Add point to list
            self.points.append((x, y))
            # Draw point on image
            cv2.circle(self.current_image, (x, y), 3, (0, 255, 0), -1)
            cv2.imshow(self.window_name, self.current_image)
            
            # If we have 2 points, calculate diameter
            if len(self.points) == 2:
                # Calculate Euclidean distance
                self.diameter = np.sqrt((self.points[1][0] - self.points[0][0])**2 + 
                                     (self.points[1][1] - self.points[0][1])**2)
                # Draw line between points
                cv2.line(self.current_image, self.points[0], self.points[1], (0, 255, 0), 2)
                cv2.imshow(self.window_name, self.current_image)
            
    def measure_ball(self, image_path):
        # Reset points and diameter
        self.points = []
        self.diameter = None
        
        # Read image
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error loading image: {image_path}")
            return None
        
        # Store original and make a copy for drawing
        self.current_image = img.copy()
        
        # Create window and set mouse callback
        cv2.namedWindow(self.window_name)
        cv2.setMouseCallback(self.window_name, self.mouse_callback)
        
        print("Click on two points to measure ball diameter. Press 'c' to cancel, 'n' to confirm measurement")
        
        while True:
            cv2.imshow(self.window_name, self.current_image)
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('c'):  # Cancel measurement
                cv2.destroyWindow(self.window_name)
                return None
            elif key == ord('n') and self.diameter is not None:  # Confirm measurement
                cv2.destroyWindow(self.window_name)
                return self.diameter

In [ ]:
import cv2

video_path = 'ball.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video file.")
else:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        cv2.imshow('Video', frame) # Display the frame in a window

        if cv2.waitKey(25) & 0xFF == ord('q'): # Press 'q' to exit
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
# Process all calibration photos
calibration_dir = 'distance_calibration_photos'
ball_measurer = BallMeasurement()

# Lists to store measurements
distances = np.array([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 11.66]) * (11.5/12)  # in feet
diameters = [314.78, 262.03, 188.0, 164.15, 143.57, 119.33, 106.08, 98.08, 86.47, 81.58, 79.10]  # in pixels

# Process each image
for filename in sorted(os.listdir(calibration_dir)):
    if filename.endswith('.jpeg'):
        # Extract distance from filename (assuming format like '10ft.jpeg')
        distance = float(filename.replace('ft.jpeg', ''))
        
        # Full path to image
        image_path = os.path.join(calibration_dir, filename)
        
        print(f"\nProcessing image at {distance} feet...")
        diameter = ball_measurer.measure_ball(image_path)
        
        if diameter is not None:
            distances.append(distance)
            diameters.append(diameter)
            print(f"Measured diameter: {diameter:.2f} pixels")

# Print results
print("\nCalibration Results:")
for dist, diam in zip(distances, diameters):
    print(f"Distance: {dist}ft, Ball diameter: {diam:.2f} pixels")

# Save results to lists for later use
distance = distances
ball_diam = diameters

In [ ]:
import matplotlib.pyplot as plt
distances = np.array([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 11.66]) * (11.5/12)  # in feet
diameters = [314.78, 262.03, 188.0, 164.15, 143.57, 119.33, 106.08, 98.08, 86.47, 81.58, 79.10]  # in pixels

# Create the plot
plt.figure(figsize=(10, 6))
plt.scatter(distances, diameters, color='blue', label='Measurements')
plt.plot(distances, diameters, 'b--', alpha=0.5)

# Add labels and title
plt.xlabel('Distance (feet)')
plt.ylabel('Ball Diameter (pixels)')
plt.title('Ball Size vs Distance Calibration')
plt.grid(True, alpha=0.3)
plt.legend()

# Add a trend line to help visualize the relationship
z = np.polyfit(distances, diameters, 2)
p = np.poly1d(z)
x_trend = np.linspace(min(distances), max(distances), 100)
plt.plot(x_trend, p(x_trend), 'r-', alpha=0.7, label=f'Trend Line (degree=2)')
plt.legend()

plt.show()

# Print the polynomial coefficients for future reference
print("\nQuadratic fit coefficients (ax² + bx + c):")
print(f"a = {z[0]:.4f}")
print(f"b = {z[1]:.4f}")
print(f"c = {z[2]:.4f}")

In [ ]:
from ultralytics import YOLO
model = YOLO("yolo11n.pt") # Load a nano-sized YOLOv11 model
results = model.predict("distance_calibration_photos/11ft.jpeg")
for r in results:
    im_array = r.plot()  # plot returns an image with detections
    im = Image.fromarray(im_array[..., ::-1])  # Convert to RGB PIL image
    im.show() # Display the image
